# Simulation Ramp

In [1]:
#-----Import essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Deine some important variables.

p_0 = 1.6;
p_1 = 1.8;
t_max  = 5.0;
ramp_steps = 1e5;

#-----Create a simulation context.

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Initialize from a gsd file.

system = hoomd.init.read_gsd("T_CM_5.0_P_1.6_ramp.gsd",frame = -1)

#-----Define each mesogen in the local frame of reference in the rigid body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Create mesogens.

rigid.create_bodies();

#-----Define the potential energy.

nl = hoomd.md.nlist.cell();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define the interaction between each constituent in the mix.

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define the group on wich we will integrate the equations of motion.

mesogens = hoomd.group.rigid_center();

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
notice(2): Group "rigid_center" created containing 1000 particles


In [2]:
#----Integrate using NPT

press_1 = hoomd.variant.linear_interp(points = [(0.0,p_0), (p_ramp, p_1)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 0.5, tauP = 1.0, P = press_1);
npt.randomize_velocities(seed = 42)

#-----Save data in a .gds and .log file.

log_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [2]:
#-----Run the simulation.

hoomd.run(ramp_steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 830088 / 930000 | TPS 8.70375 | ETA 03:11:19
Time 00:00:20 | Step 830183 / 930000 | TPS 9.39964 | ETA 02:56:59
Time 00:00:30 | Step 830283 / 930000 | TPS 9.99079 | ETA 02:46:20
Time 00:00:40 | Step 830377 / 930000 | TPS 9.34702 | ETA 02:57:38
Time 00:00:50 | Step 830476 / 930000 | TPS 9.83871 | ETA 02:48:35
Time 00:01:00 | Step 830573 / 930000 | TPS 9.67267 | ETA 02:51:19
Time 00:01:10 | Step 830668 / 930000 | TPS 9.48973 | ETA 02:54:27
Time 00:01:20 | Step 830768 / 930000 | TPS 9.96984 | ETA 02:45:53
Time 00:01:30 | Step 830864 / 930000 | TPS 9.57424 | ETA 02:52:34
Time 00:01:40 | Step 830961 / 930000 | TPS 9.56696 | ETA 02:52:32
Time 00:01:50 | Step 831057 / 930000 | TPS 9.52172 | ETA 02:53:11
Time 00:02:00 | Step 831151 / 9

**ERROR**: Particle with unique tag 7524 has NaN for its position.


RuntimeError: Error computing cell list

In [ ]:
#------Display some information (density).

system.get_metadata()